# Cell Admixture Tissue Feature Reduction Utilizing Support Vector Regression Recursive Feature Elimination

This notebook is meant to encompass the entire machine learning pipeline, from initial features to selected features.

1. [Import required packages and display version information](#imports-and-version-information) 
2. [Initial feature reduction](#initial-feature-reduction) 
    1. [Load data](#load-data)
        * Initial feature lists
        * CPTAC 
        * HGSOC 
        * Celladmixture tissue data
    2. [Split tissue data into test/train](#split-tissue-data-into-test-and-train-sets)
    3. [Hyperparameter tuning](#hyperparameter-tuning)
    4. [Perform RFE on reduced CPTAC + HGSOC + Protein Candidate features](#perform-rfe)
        * SVM SVR
        * Save model performances
    5. [Choose reduced featuresets](#choose-reduced-featuresets)
        * at least 15 features, lowest test MSE
3. [Validation](#validation)
    1. [CPTAC](#cptac)
        * model results
        * ssGSEA
    2. [N9 HGSOC](#hgsoc)
        * model results
        * ssGSEA

# Imports and Version Information<a id='imports-and-version-information'></a>

In [32]:
# Dataframes, Linear Algebra, Statistics
import pandas as pd
import numpy as np

# Graphing
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
import matplotlib as mpl
from seaborn.external.appdirs import system

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from matplotlib.legend_handler import HandlerTuple
from statannotations.Annotator import Annotator
import matplotlib as mpl

# Progress Bar
from tqdm import tqdm

# System
import os
import pickle
import subprocess
import json

# download data
import requests


# more graphing stuff
mpl.rcParams["savefig.dpi"] = 350 # dpi for saving figures
figure_path = 'figures' # folder for figures (will be created if it doesn't exist)
file_format = 'png' # figure file format
if figure_path not in os.listdir(): # if the figure folder filepath doesn't exist, make the folder
    print(f'Making `./{figure_path}`.')
    os.mkdir(figure_path)
else: # else, do nothing
    print(f'`./{figure_path}` already exists.')

font = {'weight' : 'normal',
    'size'   : 15}
mpl.rc('font', **font)

# data folder
data_folder = 'data'

`./figures` already exists.


In [33]:
print('Version information:')
! pip list | grep -E 'pandas|numpy|scipy|matplotlib|seaborn|scikit-learn'

Version information:
matplotlib                3.10.0
matplotlib-inline         0.1.6
numpy                     1.26.4
pandas                    2.0.3
scikit-learn              1.6.1
scipy                     1.15.2
seaborn                   0.13.2


# Initial Feature Reduction<a id='initial-feature-reduction'></a>
2. [Initial feature reduction](#initial-feature-reduction) 
    1. [Load data](#load-data)
        * Initial feature lists
        * CPTAC 
        * HGSOC 
        * Celladmixture tissue data
    2. [Split tissue data into test/train](#split-tissue-data-into-test-and-train-sets)
    3. [Hyperparameter tuning](#hyperparameter-tuning)
    4. [Perform RFE on reduced CPTAC + HGSOC + Protein Candidate features](#perform-rfe)
        * SVM SVR
        * Save model performances
    5. [Choose reduced featuresets](#choose-reduced-featuresets)
        * at least 15 features, lowest test MSE

## Load data<a id='load-data'></a>

### Protein Candidates


In [34]:
from pathlib import Path
DATASET = "PXD023904"
DATA_FOLDER = Path(f'/Users/henning/PycharmProjects/CelltypeDeconvolution/data_bulk_processed/{DATASET}_ProteoMixture')


tls_strings = ["Agressive","Non-agressive","Vertical growth phase","Horizontal growth phase","In situ","ECM","Melanocyte"]
tls_colors = {
    "Agressive": "#FF0000",
    "Non-agressive": "#FF7F00",
    "Vertical growth phase": "#FFFF00",
    "Horizontal growth phase": "#7FFF00",
    "In situ": "#00FF00",
    "ECM": "#00FFFF",
    "Melanocyte": "#0000FF"
}
protein_candidates = {}
file_path = Path(f'{DATA_FOLDER}/{DATASET}_ProteoMixture_protein_candidates.xlsx')
if not file_path.exists():
    print(file_path.absolute())
for cell_type in tls_strings:
    protein_candidates[cell_type] = pd.read_excel(file_path)


### Tissue Data

In [35]:
synthetic_bulk_protein_data = pd.read_csv(f'{DATA_FOLDER}/{DATASET}_ProteoMixture_bulk_protein_data.csv', index_col=0)
synthetic_bulk_protein_data.fillna(0, inplace=True)
print('Admixture Abundances dataframe shape:', synthetic_bulk_protein_data.shape)
synthetic_bulk_protein_data

Admixture Abundances dataframe shape: (3861, 100)


,0_Agressive:0.1_Non-agressive:0.1_Vertical growth phase:0.0_Horizontal growth phase:0.1_In situ:0.3_ECM:0.3_Melanocyte:0.1,10_Agressive:0.2_Non-agressive:0.0_Vertical growth phase:0.2_Horizontal growth phase:0.1_In situ:0.1_ECM:0.3_Melanocyte:0.1,11_Agressive:0.1_Non-agressive:0.2_Vertical growth phase:0.0_Horizontal growth phase:0.1_In situ:0.3_ECM:0.2_Melanocyte:0.1,12_Agressive:0.0_Non-agressive:0.0_Vertical growth phase:0.1_Horizontal growth phase:0.2_In situ:0.4_ECM:0.2_Melanocyte:0.1,13_Agressive:0.0_Non-agressive:0.3_Vertical growth phase:0.0_Horizontal growth phase:0.1_In situ:0.1_ECM:0.4_Melanocyte:0.1,14_Agressive:0.2_Non-agressive:0.1_Vertical growth phase:0.0_Horizontal growth phase:0.2_In situ:0.0_ECM:0.3_Melanocyte:0.2,15_Agressive:0.2_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.1_In situ:0.3_ECM:0.1_Melanocyte:0.0,16_Agressive:0.0_Non-agressive:0.0_Vertical growth phase:0.2_Horizontal growth phase:0.1_In situ:0.4_ECM:0.3_Melanocyte:0.0,17_Agressive:0.1_Non-agressive:0.0_Vertical growth phase:0.0_Horizontal growth phase:0.3_In situ:0.1_ECM:0.3_Melanocyte:0.2,18_Agressive:0.0_Non-agressive:0.1_Vertical growth phase:0.0_Horizontal growth phase:0.0_In situ:0.5_ECM:0.2_Melanocyte:0.2,...,91_Agressive:0.0_Non-agressive:0.1_Vertical growth phase:0.0_Horizontal growth phase:0.2_In situ:0.3_ECM:0.4_Melanocyte:0.0,92_Agressive:0.1_Non-agressive:0.0_Vertical growth phase:0.2_Horizontal growth phase:0.3_In situ:0.1_ECM:0.2_Melanocyte:0.1,93_Agressive:0.0_Non-agressive:0.1_Vertical growth phase:0.1_Horizontal growth phase:0.2_In situ:0.3_ECM:0.2_Melanocyte:0.1,94_Agressive:0.1_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.1_In situ:0.1_ECM:0.2_Melanocyte:0.2,95_Agressive:0.3_Non-agressive:0.2_Vertical growth phase:0.0_Horizontal growth phase:0.2_In situ:0.0_ECM:0.2_Melanocyte:0.1,96_Agressive:0.1_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.2_In situ:0.2_ECM:0.1_Melanocyte:0.1,97_Agressive:0.2_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.1_In situ:0.1_ECM:0.1_Melanocyte:0.2,98_Agressive:0.2_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.2_In situ:0.2_ECM:0.1_Melanocyte:0.0,99_Agressive:0.1_Non-agressive:0.1_Vertical growth phase:0.3_Horizontal growth phase:0.1_In situ:0.2_ECM:0.0_Melanocyte:0.2,9_Agressive:0.2_Non-agressive:0.0_Vertical growth phase:0.1_Horizontal growth phase:0.1_In situ:0.1_ECM:0.3_Melanocyte:0.2
Accession,,,,,,,,,,,,,,,,,,,,,
A0A075B6H9,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471,...,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471
A0A075B6K5,9.537964,9.004027,9.024701,9.286203,9.533647,9.348841,8.603150,9.193691,9.386834,9.326034,...,9.686083,9.380665,9.070801,9.616763,9.522848,8.900245,9.400797,9.387640,8.931348,9.360246
A0A075B6P5,10.104309,10.291399,10.300591,10.176972,10.382731,10.326753,9.998866,10.241024,10.245919,10.328865,...,10.209660,10.444458,10.139141,10.498473,10.342013,10.137919,10.669992,10.375896,10.466862,10.389327
A0A075B6S5,8.668337,8.744482,8.239299,8.896735,8.714269,9.039145,8.368214,8.774381,8.892579,8.295426,...,8.780991,8.674322,8.239299,9.217749,8.854630,8.368214,8.584607,8.401397,9.363609,9.255350
A0A0A0MRZ8,12.108458,11.936485,11.790300,11.869556,12.346982,12.088359,11.235709,11.983771,12.006318,12.039146,...,12.098020,11.947155,11.901252,12.349592,11.915973,11.313102,11.819399,11.596826,11.695561,12.201189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q9Y6V7,8.052041,8.052041,8.052041,8.145617,8.233492,8.342906,8.204791,8.175508,8.342906,8.175508,...,8.204791,8.342906,8.145617,8.052041,8.204791,8.052041,8.204791,8.052041,8.052041,8.342906
Q9Y6W3,8.876849,9.036653,9.095982,9.196590,8.444298,8.910369,8.931782,9.157815,8.961108,9.056050,...,8.864341,9.135342,9.236530,9.081850,8.643793,8.594517,8.566971,8.400693,

In [36]:
admixture_tissue_annotation = synthetic_bulk_protein_data.copy().drop(columns=filter(lambda x: 'Accession' not in x, synthetic_bulk_protein_data.columns))
admixture_tissue_annotation['Gene'] = ""
admixture_tissue_annotation['Description'] = ""
admixture_tissue_annotation['seq.txt'] = ""
print('Admixture Annotation dataframe shape:', admixture_tissue_annotation.shape)
admixture_tissue_annotation

Admixture Annotation dataframe shape: (3861, 3)


,Gene,Description,seq.txt
Accession,,,
A0A075B6H9,,,
A0A075B6K5,,,
A0A075B6P5,,,
A0A075B6S5,,,
A0A0A0MRZ8,,,
...,...,...,...
Q9Y6V7,,,
Q9Y6W3,,,
Q9Y6W5,,,


### Get Final Dataset (overlaps)

In [37]:
# load the metadata
synthetic_bulk_metadata = pd.read_csv(f'{DATA_FOLDER}/{DATASET}_ProteoMixture_bulk_metadata.csv')
synthetic_bulk_metadata = synthetic_bulk_metadata.T
synthetic_bulk_metadata.columns = synthetic_bulk_metadata.iloc[0]
synthetic_bulk_metadata = synthetic_bulk_metadata[1:]
synthetic_bulk_metadata

Sample,0_Agressive:0.1_Non-agressive:0.1_Vertical growth phase:0.0_Horizontal growth phase:0.1_In situ:0.3_ECM:0.3_Melanocyte:0.1,1_Agressive:0.0_Non-agressive:0.3_Vertical growth phase:0.2_Horizontal growth phase:0.1_In situ:0.2_ECM:0.2_Melanocyte:0.0,2_Agressive:0.1_Non-agressive:0.1_Vertical growth phase:0.1_Horizontal growth phase:0.0_In situ:0.2_ECM:0.2_Melanocyte:0.3,3_Agressive:0.2_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.0_In situ:0.2_ECM:0.3_Melanocyte:0.0,4_Agressive:0.1_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.1_In situ:0.2_ECM:0.1_Melanocyte:0.2,5_Agressive:0.0_Non-agressive:0.2_Vertical growth phase:0.0_Horizontal growth phase:0.2_In situ:0.2_ECM:0.3_Melanocyte:0.1,6_Agressive:0.0_Non-agressive:0.1_Vertical growth phase:0.1_Horizontal growth phase:0.3_In situ:0.2_ECM:0.2_Melanocyte:0.1,7_Agressive:0.3_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.0_In situ:0.2_ECM:0.1_Melanocyte:0.1,8_Agressive:0.1_Non-agressive:0.1_Vertical growth phase:0.1_Horizontal growth phase:0.2_In situ:0.2_ECM:0.1_Melanocyte:0.2,9_Agressive:0.2_Non-agressive:0.0_Vertical growth phase:0.1_Horizontal growth phase:0.1_In situ:0.1_ECM:0.3_Melanocyte:0.2,...,90_Agressive:0.0_Non-agressive:0.1_Vertical growth phase:0.0_Horizontal growth phase:0.1_In situ:0.2_ECM:0.4_Melanocyte:0.2,91_Agressive:0.0_Non-agressive:0.1_Vertical growth phase:0.0_Horizontal growth phase:0.2_In situ:0.3_ECM:0.4_Melanocyte:0.0,92_Agressive:0.1_Non-agressive:0.0_Vertical growth phase:0.2_Horizontal growth phase:0.3_In situ:0.1_ECM:0.2_Melanocyte:0.1,93_Agressive:0.0_Non-agressive:0.1_Vertical growth phase:0.1_Horizontal growth phase:0.2_In situ:0.3_ECM:0.2_Melanocyte:0.1,94_Agressive:0.1_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.1_In situ:0.1_ECM:0.2_Melanocyte:0.2,95_Agressive:0.3_Non-agressive:0.2_Vertical growth phase:0.0_Horizontal growth phase:0.2_In situ:0.0_ECM:0.2_Melanocyte:0.1,96_Agressive:0.1_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.2_In situ:0.2_ECM:0.1_Melanocyte:0.1,97_Agressive:0.2_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.1_In situ:0.1_ECM:0.1_Melanocyte:0.2,98_Agressive:0.2_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.2_In situ:0.2_ECM:0.1_Melanocyte:0.0,99_Agressive:0.1_Non-agressive:0.1_Vertical growth phase:0.3_Horizontal growth phase:0.1_In situ:0.2_ECM:0.0_Melanocyte:0.2
Agressive,0.1,0.0,0.1,0.2,0.1,0.0,0.0,0.3,0.1,0.2,...,0.0,0.0,0.1,0.0,0.1,0.3,0.1,0.2,0.2,0.1
Non-agressive,0.1,0.3,0.1,0.2,0.2,0.2,0.1,0.2,0.1,0.0,...,0.1,0.1,0.0,0.1,0.2,0.2,0.2,0.2,0.2,0.1
Vertical growth phase,0.0,0.2,0.1,0.1,0.1,0.0,0.1,0.1,0.1,0.1,...,0.0,0.0,0.2,0.1,0.1,0.0,0.1,0.1,0.1,0.3
Horizontal growth phase,0.1,0.1,0.0,0.0,0.1,0.2,0.3,0.0,0.2,0.1,...,0.1,0.2,0.3,0.2,0.1,0.2,0.2,0.1,0.2,0.1
In situ,0.3,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.1,...,0.2,0.3,0.1,0.3,0.1,0.0,0.2,0.1,0.2,0.2
ECM,0.3,0.2,0.2,0.3,0.1,0.3,0.2,0.1,0.1,0.3,...,0.4,0.4,0.2,0.2,0.2,0.2,0.1,0.1,0.1,0.0
Melanocyte,0.1,0.0,0.3,0.0,0.2,0.1,0.1,0.1,0.2,0.2,...,0.2,0.0,0.1,0.1,0.2,0.1,0.1,0.2,0.0,0.2


In [38]:
print('Original abundances shape:', synthetic_bulk_protein_data)
# hgsoc overlap
# admixture_tissue_log2_imputed_abundances_hgsoc_overlap = admixture_tissue_log2_imputed_abundances[admixture_tissue_log2_imputed_abundances.index.isin(hgsoc_abundance.columns.values)]
# print('Overlap with n9 HGSOC shape:', admixture_tissue_log2_imputed_abundances_hgsoc_overlap.shape)
# cptac overlap
# final_admixture_tissue_log2_imputed_abundances = admixture_tissue_log2_imputed_abundances_hgsoc_overlap[admixture_tissue_log2_imputed_abundances_hgsoc_overlap.index.isin(cptac_accession_numbers.columns.values)]
# print('Overlap with n9 HGSOC and CPTAC (final) shape:', final_admixture_tissue_log2_imputed_abundances.shape)

# in concordance with the originally provided notebook
final_admixture_tissue_log2_imputed_abundances = synthetic_bulk_protein_data
final_admixture_tissue_log2_imputed_abundances


Original abundances shape:             0_Agressive:0.1_Non-agressive:0.1_Vertical growth phase:0.0_Horizontal growth phase:0.1_In situ:0.3_ECM:0.3_Melanocyte:0.1  \
Accession                                                                                                                                
A0A075B6H9                                          10.942471                                                                            
A0A075B6K5                                           9.537964                                                                            
A0A075B6P5                                          10.104309                                                                            
A0A075B6S5                                           8.668337                                                                            
A0A0A0MRZ8                                          12.108458                                                                            
...    

,0_Agressive:0.1_Non-agressive:0.1_Vertical growth phase:0.0_Horizontal growth phase:0.1_In situ:0.3_ECM:0.3_Melanocyte:0.1,10_Agressive:0.2_Non-agressive:0.0_Vertical growth phase:0.2_Horizontal growth phase:0.1_In situ:0.1_ECM:0.3_Melanocyte:0.1,11_Agressive:0.1_Non-agressive:0.2_Vertical growth phase:0.0_Horizontal growth phase:0.1_In situ:0.3_ECM:0.2_Melanocyte:0.1,12_Agressive:0.0_Non-agressive:0.0_Vertical growth phase:0.1_Horizontal growth phase:0.2_In situ:0.4_ECM:0.2_Melanocyte:0.1,13_Agressive:0.0_Non-agressive:0.3_Vertical growth phase:0.0_Horizontal growth phase:0.1_In situ:0.1_ECM:0.4_Melanocyte:0.1,14_Agressive:0.2_Non-agressive:0.1_Vertical growth phase:0.0_Horizontal growth phase:0.2_In situ:0.0_ECM:0.3_Melanocyte:0.2,15_Agressive:0.2_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.1_In situ:0.3_ECM:0.1_Melanocyte:0.0,16_Agressive:0.0_Non-agressive:0.0_Vertical growth phase:0.2_Horizontal growth phase:0.1_In situ:0.4_ECM:0.3_Melanocyte:0.0,17_Agressive:0.1_Non-agressive:0.0_Vertical growth phase:0.0_Horizontal growth phase:0.3_In situ:0.1_ECM:0.3_Melanocyte:0.2,18_Agressive:0.0_Non-agressive:0.1_Vertical growth phase:0.0_Horizontal growth phase:0.0_In situ:0.5_ECM:0.2_Melanocyte:0.2,...,91_Agressive:0.0_Non-agressive:0.1_Vertical growth phase:0.0_Horizontal growth phase:0.2_In situ:0.3_ECM:0.4_Melanocyte:0.0,92_Agressive:0.1_Non-agressive:0.0_Vertical growth phase:0.2_Horizontal growth phase:0.3_In situ:0.1_ECM:0.2_Melanocyte:0.1,93_Agressive:0.0_Non-agressive:0.1_Vertical growth phase:0.1_Horizontal growth phase:0.2_In situ:0.3_ECM:0.2_Melanocyte:0.1,94_Agressive:0.1_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.1_In situ:0.1_ECM:0.2_Melanocyte:0.2,95_Agressive:0.3_Non-agressive:0.2_Vertical growth phase:0.0_Horizontal growth phase:0.2_In situ:0.0_ECM:0.2_Melanocyte:0.1,96_Agressive:0.1_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.2_In situ:0.2_ECM:0.1_Melanocyte:0.1,97_Agressive:0.2_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.1_In situ:0.1_ECM:0.1_Melanocyte:0.2,98_Agressive:0.2_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.2_In situ:0.2_ECM:0.1_Melanocyte:0.0,99_Agressive:0.1_Non-agressive:0.1_Vertical growth phase:0.3_Horizontal growth phase:0.1_In situ:0.2_ECM:0.0_Melanocyte:0.2,9_Agressive:0.2_Non-agressive:0.0_Vertical growth phase:0.1_Horizontal growth phase:0.1_In situ:0.1_ECM:0.3_Melanocyte:0.2
Accession,,,,,,,,,,,,,,,,,,,,,
A0A075B6H9,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471,...,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471,10.942471
A0A075B6K5,9.537964,9.004027,9.024701,9.286203,9.533647,9.348841,8.603150,9.193691,9.386834,9.326034,...,9.686083,9.380665,9.070801,9.616763,9.522848,8.900245,9.400797,9.387640,8.931348,9.360246
A0A075B6P5,10.104309,10.291399,10.300591,10.176972,10.382731,10.326753,9.998866,10.241024,10.245919,10.328865,...,10.209660,10.444458,10.139141,10.498473,10.342013,10.137919,10.669992,10.375896,10.466862,10.389327
A0A075B6S5,8.668337,8.744482,8.239299,8.896735,8.714269,9.039145,8.368214,8.774381,8.892579,8.295426,...,8.780991,8.674322,8.239299,9.217749,8.854630,8.368214,8.584607,8.401397,9.363609,9.255350
A0A0A0MRZ8,12.108458,11.936485,11.790300,11.869556,12.346982,12.088359,11.235709,11.983771,12.006318,12.039146,...,12.098020,11.947155,11.901252,12.349592,11.915973,11.313102,11.819399,11.596826,11.695561,12.201189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q9Y6V7,8.052041,8.052041,8.052041,8.145617,8.233492,8.342906,8.204791,8.175508,8.342906,8.175508,...,8.204791,8.342906,8.145617,8.052041,8.204791,8.052041,8.204791,8.052041,8.052041,8.342906
Q9Y6W3,8.876849,9.036653,9.095982,9.196590,8.444298,8.910369,8.931782,9.157815,8.961108,9.056050,...,8.864341,9.135342,9.236530,9.081850,8.643793,8.594517,8.566971,8.400693,

## Split Tissue Data Into Test and Train Sets<a id='split-tissue-data-into-test-and-train-sets'></a>

There are 16 samples, so 10 will be used for training, and 6 for testing. Additionally, test/train sets will be decided for each tissue subtype, and will be stratified to a feature generated by the amount of that type of tissue (giving test and train sets similar distribution of tissue %).

### Get features from sample names

In [39]:
# in concordance with the originally provided notebook
target = synthetic_bulk_metadata
target

Sample,0_Agressive:0.1_Non-agressive:0.1_Vertical growth phase:0.0_Horizontal growth phase:0.1_In situ:0.3_ECM:0.3_Melanocyte:0.1,1_Agressive:0.0_Non-agressive:0.3_Vertical growth phase:0.2_Horizontal growth phase:0.1_In situ:0.2_ECM:0.2_Melanocyte:0.0,2_Agressive:0.1_Non-agressive:0.1_Vertical growth phase:0.1_Horizontal growth phase:0.0_In situ:0.2_ECM:0.2_Melanocyte:0.3,3_Agressive:0.2_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.0_In situ:0.2_ECM:0.3_Melanocyte:0.0,4_Agressive:0.1_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.1_In situ:0.2_ECM:0.1_Melanocyte:0.2,5_Agressive:0.0_Non-agressive:0.2_Vertical growth phase:0.0_Horizontal growth phase:0.2_In situ:0.2_ECM:0.3_Melanocyte:0.1,6_Agressive:0.0_Non-agressive:0.1_Vertical growth phase:0.1_Horizontal growth phase:0.3_In situ:0.2_ECM:0.2_Melanocyte:0.1,7_Agressive:0.3_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.0_In situ:0.2_ECM:0.1_Melanocyte:0.1,8_Agressive:0.1_Non-agressive:0.1_Vertical growth phase:0.1_Horizontal growth phase:0.2_In situ:0.2_ECM:0.1_Melanocyte:0.2,9_Agressive:0.2_Non-agressive:0.0_Vertical growth phase:0.1_Horizontal growth phase:0.1_In situ:0.1_ECM:0.3_Melanocyte:0.2,...,90_Agressive:0.0_Non-agressive:0.1_Vertical growth phase:0.0_Horizontal growth phase:0.1_In situ:0.2_ECM:0.4_Melanocyte:0.2,91_Agressive:0.0_Non-agressive:0.1_Vertical growth phase:0.0_Horizontal growth phase:0.2_In situ:0.3_ECM:0.4_Melanocyte:0.0,92_Agressive:0.1_Non-agressive:0.0_Vertical growth phase:0.2_Horizontal growth phase:0.3_In situ:0.1_ECM:0.2_Melanocyte:0.1,93_Agressive:0.0_Non-agressive:0.1_Vertical growth phase:0.1_Horizontal growth phase:0.2_In situ:0.3_ECM:0.2_Melanocyte:0.1,94_Agressive:0.1_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.1_In situ:0.1_ECM:0.2_Melanocyte:0.2,95_Agressive:0.3_Non-agressive:0.2_Vertical growth phase:0.0_Horizontal growth phase:0.2_In situ:0.0_ECM:0.2_Melanocyte:0.1,96_Agressive:0.1_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.2_In situ:0.2_ECM:0.1_Melanocyte:0.1,97_Agressive:0.2_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.1_In situ:0.1_ECM:0.1_Melanocyte:0.2,98_Agressive:0.2_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.2_In situ:0.2_ECM:0.1_Melanocyte:0.0,99_Agressive:0.1_Non-agressive:0.1_Vertical growth phase:0.3_Horizontal growth phase:0.1_In situ:0.2_ECM:0.0_Melanocyte:0.2
Agressive,0.1,0.0,0.1,0.2,0.1,0.0,0.0,0.3,0.1,0.2,...,0.0,0.0,0.1,0.0,0.1,0.3,0.1,0.2,0.2,0.1
Non-agressive,0.1,0.3,0.1,0.2,0.2,0.2,0.1,0.2,0.1,0.0,...,0.1,0.1,0.0,0.1,0.2,0.2,0.2,0.2,0.2,0.1
Vertical growth phase,0.0,0.2,0.1,0.1,0.1,0.0,0.1,0.1,0.1,0.1,...,0.0,0.0,0.2,0.1,0.1,0.0,0.1,0.1,0.1,0.3
Horizontal growth phase,0.1,0.1,0.0,0.0,0.1,0.2,0.3,0.0,0.2,0.1,...,0.1,0.2,0.3,0.2,0.1,0.2,0.2,0.1,0.2,0.1
In situ,0.3,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.1,...,0.2,0.3,0.1,0.3,0.1,0.0,0.2,0.1,0.2,0.2
ECM,0.3,0.2,0.2,0.3,0.1,0.3,0.2,0.1,0.1,0.3,...,0.4,0.4,0.2,0.2,0.2,0.2,0.1,0.1,0.1,0.0
Melanocyte,0.1,0.0,0.3,0.0,0.2,0.1,0.1,0.1,0.2,0.2,...,0.2,0.0,0.1,0.1,0.2,0.1,0.1,0.2,0.0,0.2


### Create features to stratify to
Creating 3 groups for each feature to try and evenly spread % values between test and train sets.

In [40]:
# create a feature that is 0, 1, or 2 depending on whether the cell type percentage is 0-24%, 25-75%, and 75-100%
target_stratify = pd.DataFrame(columns=synthetic_bulk_metadata.columns)
target_stratify = target_stratify.T

for tissue_type in synthetic_bulk_metadata.index:
    temp = []
    for tissue_percentage, sample in zip(target.T[tissue_type], target.columns):
        if tissue_percentage < 0.10:
            temp.append(0)
        elif tissue_percentage >= 0.15 and tissue_percentage < 0.20:
            temp.append(1)
        else:
            temp.append(2)
    target_stratify[tissue_type] = temp
target_stratify

,Agressive,Non-agressive,Vertical growth phase,Horizontal growth phase,In situ,ECM,Melanocyte
Sample,,,,,,,
0_Agressive:0.1_Non-agressive:0.1_Vertical growth phase:0.0_Horizontal growth phase:0.1_In situ:0.3_ECM:0.3_Melanocyte:0.1,2,2,0,2,2,2,2
1_Agressive:0.0_Non-agressive:0.3_Vertical growth phase:0.2_Horizontal growth phase:0.1_In situ:0.2_ECM:0.2_Melanocyte:0.0,0,2,2,2,2,2,0
2_Agressive:0.1_Non-agressive:0.1_Vertical growth phase:0.1_Horizontal growth phase:0.0_In situ:0.2_ECM:0.2_Melanocyte:0.3,2,2,2,0,2,2,2
3_Agressive:0.2_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.0_In situ:0.2_ECM:0.3_Melanocyte:0.0,2,2,2,0,2,2,0
4_Agressive:0.1_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.1_In situ:0.2_ECM:0.1_Melanocyte:0.2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...
95_Agressive:0.3_Non-agressive:0.2_Vertical growth phase:0.0_Horizontal growth phase:0.2_In situ:0.0_ECM:0.2_Melanocyte:0.1,2,2,0,2,0,2,2
96_Agressive:0.1_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.2_In situ:0.2_ECM:0.1_Melanocyte:0.1,2,2,2,2,2,2,2
97_Agressive:0.2_Non-agressive:0.2_Vertical growth phase:0.1_Horizontal growth phase:0.1_In situ:0.1_ECM:0.1_Melanocyte:0.2,2,2,2,2,2,2,2


### Create test/train splits for each target
Essentially three separate models will be made. They will eventually be combined to create a predictor.

In [41]:
# create directories to hold test/train data
for tissue_type in tls_strings:
    try:
        os.mkdir(f'{data_folder}/{tissue_type}')
        print(f'Created folder at `{data_folder}/{tissue_type}`')
    except FileExistsError:
        print(f'`{data_folder}/{tissue_type}` already exists.')

`data/Agressive` already exists.
`data/Non-agressive` already exists.
`data/Vertical growth phase` already exists.
`data/Horizontal growth phase` already exists.
`data/In situ` already exists.
`data/ECM` already exists.
`data/Melanocyte` already exists.


In [42]:
final_admixture_tissue_log2_imputed_abundances = synthetic_bulk_protein_data
print('CPTAC/HGSOC abundance overlap shape:', final_admixture_tissue_log2_imputed_abundances.shape)
for tissue_type in tls_strings:
    print(f'Making train/test for {tissue_type}...')

    # overlap with protein candidates!!!
    final_protein_candidate_data = final_admixture_tissue_log2_imputed_abundances[final_admixture_tissue_log2_imputed_abundances.index.isin(protein_candidates[tissue_type]['UniProt Accession'])]
    print(f'Final CPTAC/HGSOC/{tissue_type} protein candidate abundance overlap shape:', final_protein_candidate_data.shape)

    # get train/test split
    X_train, X_test, y_train, y_test = train_test_split(final_protein_candidate_data.T, target.T[[tissue_type]], test_size=0.33, random_state=42, stratify=target_stratify[[tissue_type]])
    
    # print sizes
    print('X_train shape:', X_train.shape)
    print('X_test shape :', X_test.shape)

    # save them in their respective folders
    print(f'Saving train and test splits in `{data_folder}/{tissue_type}/`')
    datasets = [('X_train', X_train), ('X_test', X_test), ('y_train', y_train), ('y_test', y_test)] 
    for name, data in datasets:
        data.to_csv(f'{data_folder}/{tissue_type}/Tissue_{name}_{tissue_type}.csv')

CPTAC/HGSOC abundance overlap shape: (3861, 100)
Making train/test for Agressive...
Final CPTAC/HGSOC/Agressive protein candidate abundance overlap shape: (1194, 100)
X_train shape: (67, 1194)
X_test shape : (33, 1194)
Saving train and test splits in `data/Agressive/`
Making train/test for Non-agressive...
Final CPTAC/HGSOC/Non-agressive protein candidate abundance overlap shape: (1194, 100)
X_train shape: (67, 1194)
X_test shape : (33, 1194)
Saving train and test splits in `data/Non-agressive/`
Making train/test for Vertical growth phase...
Final CPTAC/HGSOC/Vertical growth phase protein candidate abundance overlap shape: (1194, 100)
X_train shape: (67, 1194)
X_test shape : (33, 1194)
Saving train and test splits in `data/Vertical growth phase/`
Making train/test for Horizontal growth phase...
Final CPTAC/HGSOC/Horizontal growth phase protein candidate abundance overlap shape: (1194, 100)
X_train shape: (67, 1194)
X_test shape : (33, 1194)
Saving train and test splits in `data/Horizon

In [43]:
# load saved data
data_sections = ['X_train', 'X_test', 'y_train', 'y_test']

data = {}

# read in data in loops 
for tissue_type in tls_strings:
    temp_tissue = {}
    for ml_section in data_sections:
        f = pd.read_csv(f'{data_folder}/{tissue_type}/Tissue_{ml_section}_{tissue_type}.csv', index_col=0)
        temp_tissue[ml_section] = f
    data[tissue_type] = temp_tissue
print('Outer-most keys:')
print(data.keys())
print('Inner keys:')
print(data[tls_strings[0]].keys())

Outer-most keys:
dict_keys(['Agressive', 'Non-agressive', 'Vertical growth phase', 'Horizontal growth phase', 'In situ', 'ECM', 'Melanocyte'])
Inner keys:
dict_keys(['X_train', 'X_test', 'y_train', 'y_test'])


In [44]:
# save data
pickle.dump(data, open(f"{data_folder}/admixture/data.pkl", 'wb'))

## Hyperparameter tuning<a id='hyperparameter-tuning'></a>

Here, I create a parameter grid, and using sklearn's GridSearchCV function, each set of parameters is used to train and test a model 5 times. The parameters that get the best average score end up being the parameters that are used to create the models that will be recursively trained in RFE.

In [45]:
# create parameter grid
svr_parameter_grid = [
    {'kernel': ['linear'], 
    'C': [0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1], 
    'epsilon': [0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]}
]
# the larger epsilon, the more error is allowed within model
# C is multiplied to the regularization 

folds = 5 # 5 fold, each test set will have 2 samples
print('Using a grid search approach to tune hyperparameters for Support Vector Regression')

svr = svm.SVR()
grid_search = GridSearchCV(svr, svr_parameter_grid, n_jobs=-1, cv=folds)

# create dictionary to hold hyperparameters for each tissue type
hyperparameters = {}

for tissue_type in tqdm(tls_strings):
    X_train, y_train = data[tissue_type]['X_train'], np.ravel(data[tissue_type]['y_train'])
    grid_search.fit(X_train, y_train)
    hyperparameters[tissue_type] = grid_search.best_params_

Using a grid search approach to tune hyperparameters for Support Vector Regression


100%|██████████| 7/7 [00:20<00:00,  2.95s/it]


In [46]:
print(f'Hyperparameters found by grid search using a {folds} fold Cross-Validation')
hyperparameters

Hyperparameters found by grid search using a 5 fold Cross-Validation


{'Agressive': {'C': 1e-05, 'epsilon': 0.1, 'kernel': 'linear'},
 'Non-agressive': {'C': 1e-07, 'epsilon': 0.01, 'kernel': 'linear'},
 'Vertical growth phase': {'C': 1e-05, 'epsilon': 0.1, 'kernel': 'linear'},
 'Horizontal growth phase': {'C': 1e-07, 'epsilon': 0.1, 'kernel': 'linear'},
 'In situ': {'C': 1e-05, 'epsilon': 0.01, 'kernel': 'linear'},
 'ECM': {'C': 1e-07, 'epsilon': 0.01, 'kernel': 'linear'},
 'Melanocyte': {'C': 1e-07, 'epsilon': 0.01, 'kernel': 'linear'}}

In [47]:
# defaults
"""
for tissue_type in tls_strings:
    for argument, value in [('C', 1), ('epsilon', 0.1), ('kernel', 'linear')]:
        hyperparameters[tissue_type][argument] = value
"""
print('Printing hyperparameters that will be used:')
hyperparameters

Printing hyperparameters that will be used:


{'Agressive': {'C': 1e-05, 'epsilon': 0.1, 'kernel': 'linear'},
 'Non-agressive': {'C': 1e-07, 'epsilon': 0.01, 'kernel': 'linear'},
 'Vertical growth phase': {'C': 1e-05, 'epsilon': 0.1, 'kernel': 'linear'},
 'Horizontal growth phase': {'C': 1e-07, 'epsilon': 0.1, 'kernel': 'linear'},
 'In situ': {'C': 1e-05, 'epsilon': 0.01, 'kernel': 'linear'},
 'ECM': {'C': 1e-07, 'epsilon': 0.01, 'kernel': 'linear'},
 'Melanocyte': {'C': 1e-07, 'epsilon': 0.01, 'kernel': 'linear'}}

## Perform RFE<a id='perform-rfe'></a>

Recursive Feature Elimination, or RFE, is used here to choose a subset of genes that hold the most predictive power for an support vector regressor (SVR). This can help reduce reduncancies when utilizing tools such as ssGSEA for gene set enrichment analyses.

RFE is performed 3 times, once for each tissue type (stroma, lymphocyte, or tumor). The ranks provided by RFE are then used to re-train models and assess their performance on test data. This is visualized with a scatterplot of the mean squared error (MSE) on the y-axis, and number of features on the x-axis. 

In [48]:
rankings = {}
model_performances = {}
best_models = {}

In [49]:
print(f'Performing RFE per Target ({tls_strings})')
for tissue_type in tls_strings:
    print('----', tissue_type, 'RFE ----')
    # define estimator based on hyperparameter tuning
    svr = svm.SVR(**hyperparameters[tissue_type])
    
    # define an RFE
    rfe = RFE(estimator=svr, n_features_to_select=1, step=1, verbose=False)

    # get train data
    X_train, y_train = data[tissue_type]['X_train'], np.ravel(data[tissue_type]['y_train'])

    # fit rfe to get ranking
    rfe.fit(X_train, y_train)
    ranking = rfe.ranking_
    rankings[tissue_type] = ranking

    # get test data
    y_test, X_test = data[tissue_type]['y_test'], data[tissue_type]['X_test']

    # fit all models at different featurelist sizes and plot accuracies
    mse_dict = {}
    models = {}
    min_mse = np.inf
    best_model = None
    print('Train models with each set of features, removing least important ones (by RFE rank):')
    for num_features in tqdm(range(X_train.shape[1], 14, -1)): # for each protein
        rank = pd.Series(ranking, index=X_train.columns).sort_values() # sort by importance rank
        
        # take the top i ranks of training data
        top_num_features = rank.iloc[:num_features]
        top_num_features = X_train[top_num_features.index]

        # fit svr to reduced featurelist training data
        svr.fit(top_num_features, np.ravel(y_train))
        
        # get predictions for reduced features
        y_pred = svr.predict(X_test[top_num_features.columns])
        
        # calculate mean squared error between predictions and actual
        mse = mean_squared_error(y_test, y_pred)

        # get the bets model
        if mse < min_mse:
            min_mse = mse
            best_model = (top_num_features.shape, top_num_features)
            # add to mse dictionary
            mse_dict[num_features] = mse

            # add model to model dictionary
            models[num_features] = (mse, top_num_features)

    pickle.dump(models, open(f'{data_folder}/{tissue_type}/{tissue_type}_model_performances.pkl', 'wb'))

    model_performances[tissue_type] = models

    #fig, ax = plt.subplots()
    #ax.set_xlabel('# Features (Genes)')
    #ax.set_ylabel('Mean Squared Error (Test Data)')
    #ax.set_title(f'Tissue: {tissue_type} - SVM SVR Model Performances')
    #best_models[tissue_type] = best_model
    #plt.scatter(x=mse_dict.keys(), y=mse_dict.values(), color=tls_colors[tissue_type])

    #plt.savefig(f'{figure_path}/tissue_{tissue_type}_svm_svr_model_performances.{file_format}', format=file_format)

    #plt.show()

Performing RFE per Target (['Agressive', 'Non-agressive', 'Vertical growth phase', 'Horizontal growth phase', 'In situ', 'ECM', 'Melanocyte'])
---- Agressive RFE ----
Train models with each set of features, removing least important ones (by RFE rank):


100%|██████████| 1180/1180 [00:20<00:00, 57.41it/s] 


---- Non-agressive RFE ----
Train models with each set of features, removing least important ones (by RFE rank):


100%|██████████| 1180/1180 [00:16<00:00, 72.46it/s] 


---- Vertical growth phase RFE ----
Train models with each set of features, removing least important ones (by RFE rank):


...

100%|██████████| 1180/1180 [00:09<00:00, 124.56it/s]


---- ECM RFE ----
Train models with each set of features, removing least important ones (by RFE rank):


100%|██████████| 1180/1180 [00:07<00:00, 166.60it/s]


---- Melanocyte RFE ----
Train models with each set of features, removing least important ones (by RFE rank):


100%|██████████| 1180/1180 [00:07<00:00, 167.57it/s]


In [50]:
# save rankings
pickle.dump(rankings, open(f'{data_folder}/admixture/rankings.pkl', 'wb'))

## Decide on Model for Reduced Featuresets<a id='choose-reduced-featuresets'></a>

To choose a subset of genes for each tissue type, the model with the lowest test MSE with at least 15 features is chosen. The features used in these models will be the reduced featurelists for tools such as ssGSEA. 

Aside: It might be worth looking into choosing models around the 'elbow' of the previous scatterplots. This is where the models don't necessarily have the lowest MSE, but have the fewest features/genes, and could further reduce redundancy while maintaining predictive power.

### Lowest MSE with at least 15 features
This ends up being just the lowest MSE models

In [51]:
min_num_features = 15

In [52]:
tls_features = {}
prediction_dataframe_parts = {}
for tissue_type in tls_strings:
    tissue_specific_model_performances = pd.DataFrame([(x, model_performances[tissue_type][x][0], model_performances[tissue_type][x][1]) for x in model_performances[tissue_type].keys()]).rename(columns={0:'num_features', 1:'mse', 2:'dfs'})
    # sort by MSE
    sorted_model_performances = tissue_specific_model_performances.sort_values(ascending = True, by='mse')
    # remove lower than 15 features, and get the number of features with the lowest MSE
    n_features = sorted_model_performances[sorted_model_performances.num_features >= min_num_features].iloc[0,:].num_features
    this_mse = sorted_model_performances[sorted_model_performances.num_features == n_features].mse.values[0]
    lowest_mse = np.min(tissue_specific_model_performances['mse'])
    print(f'---- {tissue_type} results ----')
    print(f'lowest test MSE of ALL models ---------------------- = {lowest_mse}')
    print(f'lowest test MSE with greater or equal to {min_num_features} features = {this_mse}')
    print(f'all features = {sorted_model_performances.shape[0]}')
    print(f'n features - = {n_features}')
    

    # get the model !
    # define estimator based on hyperparameter tuning
    svr = svm.SVR(**hyperparameters[tissue_type])

    # get train data
    y_train = np.ravel(data[tissue_type]['y_train'])
    X_train = model_performances[tissue_type][n_features][1]
    

    X_train.columns = X_train.columns.astype(str)

    # get test data
    X_test, y_test = data[tissue_type]['X_test'], np.ravel(data[tissue_type]['y_test'])

    # fit svr
    svr.fit(X_train, y_train)

    # predict svr
    y_pred = svr.predict(X_test[X_train.columns])
    prediction_dataframe_parts[tissue_type] = pd.DataFrame(y_pred, index=X_test.index, columns=[tissue_type])
    #print(f'Predictions shape: {prediction_dataframe_parts[tissue_type].shape}')
    #print(f'Predictions index: {prediction_dataframe_parts[tissue_type].index}')

    assert this_mse == mean_squared_error(y_test, y_pred), 'LOWEST MSES DO NOT MATCH' # check for reproducibility

    model_location = f'{data_folder}/{tissue_type}/{tissue_type}_svr_sklearn_at_least_{min_num_features}_model_cptac_candidate_{DATASET}.pkl'
    print(f'Writing pickled object file to `{model_location}`')
    pickle.dump(svr, open(model_location, 'wb'))

    # get features
    tls_features[tissue_type] = list(X_train.columns)

# combine predictions
predictions = pd.concat(prediction_dataframe_parts.values(), axis=1)
# sort by index
predictions = predictions.sort_index(axis=0).T.reset_index()
predictions.rename(columns={'index':'Cell Type'}, inplace=True)
print(predictions.shape)
print(predictions.columns)
print(predictions.head(10))
predictions.to_csv(f'../../deconvolution_results/{DATASET}_ProteoMixture.csv', index=False)


---- Agressive results ----
lowest test MSE of ALL models ---------------------- = 0.006665910385348851
lowest test MSE with greater or equal to 15 features = 0.006665910385348851
all features = 155
n features - = 234
Writing pickled object file to `data/Agressive/Agressive_svr_sklearn_at_least_15_model_cptac_candidate_PXD023904.pkl`
---- Non-agressive results ----
lowest test MSE of ALL models ---------------------- = 0.005918096649752929
lowest test MSE with greater or equal to 15 features = 0.005918096649752929
all features = 24
n features - = 486
Writing pickled object file to `data/Non-agressive/Non-agressive_svr_sklearn_at_least_15_model_cptac_candidate_PXD023904.pkl`
---- Vertical growth phase results ----
lowest test MSE of ALL models ---------------------- = 0.008470494716335777
lowest test MSE with greater or equal to 15 features = 0.008470494716335777
all features = 3
n features - = 1053
Writing pickled object file to `data/Vertical growth phase/Vertical growth phase_svr_skl

In [53]:
# save json file with features for each model
# serializing
json_object = json.dumps(tls_features, indent=4)

# write file
with open(f'{data_folder}/admixture/tls_gene_sets.json', 'w') as f:
    f.write(json_object)

# Validation<a id='validation'></a>
3. [Validation](#validation)
    1. [CPTAC](#cptac)
        * model results
        * ssGSEA
    2. [N9 HGSOC](#hgsoc)
        * model results
        * ssGSEA

In [54]:
model_type = f'At Least {min_num_features} Features, Lowest MSE'

In [62]:
chosen_models = {}
for tissue_type in tls_strings:
    model_location = f'{data_folder}/{tissue_type}/{tissue_type}_svr_sklearn_at_least_{min_num_features}_model_cptac_candidate_{DATASET}.pkl'
    chosen_models[tissue_type] = pickle.load(open(model_location, 'rb'))

### Estimator function
Defined here is the function that takes 3 provided models and outputs a value for each.

Negative model outputs are clipped to 0 (negative values are set to 0), and then the model outputs are each divided by their sum (they all add to 1). This is done so that the model outputs mimic % sample values.

In [63]:
# create a function that takes X as input, and outputs normalized values from estimators
def run_estimators(X: pd.DataFrame, models: dict):
    """
    A function that takes normalized abundance values and outputs normalized tissue scores for each cell type.

    Parameters:
    X: pd.DataFrame - Input data with features.
    models: dict - Dictionary where keys are cell types and values are the corresponding trained models.

    Returns:
    pd.DataFrame - Predictions for each cell type, normalized to sum to 1.
    """
    
    # ensure that X has all the required accession numbers
    accession_numbers = np.unique([feature for model in models.values() for feature in model.feature_names_in_])

    # predictions
    predictions = {}

    # make predictions
    for tissue_type, model in models.items():
        tX = X.T[X.columns.isin(model.feature_names_in_)].T
        # make missing features 0
        missing_features = list(set(model.feature_names_in_) - set(X.columns))
        if missing_features:
            print(f'{tissue_type} has missing features (all values set to 0): {missing_features}')
            print(f'{len(missing_features)/len(model.feature_names_in_) * 100:.2f}% missing for {tissue_type} model...')
            for missing_feature in missing_features:
                tX[missing_feature] = 0
        
        # predict
        tX = tX[model.feature_names_in_]
        tX.columns = tX.columns.astype(str)

        assert (tX.columns == model.feature_names_in_).all() 

        predictions[tissue_type] = model.predict(X=tX)

    # clip values lower than 0 and higher than 1
    predictions_df = np.clip(pd.DataFrame(predictions, index=X.index), 0, 1)

    # make sure values add to 1
    predictions_df = (predictions_df.T / predictions_df.sum(axis=1)).T

    return predictions_df


In [64]:
predictions_df = run_estimators(X=final_admixture_tissue_log2_imputed_abundances.T, models=chosen_models)
print('Final predictions shape:', predictions_df.shape)
predictions_df.head(100)
predictions = predictions_df.T
predictions = predictions.reset_index()
predictions.rename(columns={'index':'Cell Type'}, inplace=True)
predictions.to_csv(f'../../deconvolution_results/{DATASET}_ProteoMixture.csv', index=False)

Final predictions shape: (100, 7)


## CPTAC<a id='cptac'></a>

Here, I apply both the SVR models created, and ssGSEA methods to a Clinical Proteomic Tumor Analysis Consortium dataset, containing proteomic data from ovarian cancer samples. This serves as an external dataset for validation, with molecular subtypes based on transcriptomics being a surrogate for tissue type. 

In [58]:
validation_name = 'CPTAC'

### Model Outputs

In [59]:
for tissue_type in chosen_models:
    print(f' --- {tissue_type} --- ')
    print(f'Number of features: {len(chosen_models[tissue_type].feature_names_in_)}')

 --- Agressive --- 
Number of features: 234
 --- Non-agressive --- 
Number of features: 486
 --- Vertical growth phase --- 
Number of features: 1053
 --- Horizontal growth phase --- 
Number of features: 796
 --- In situ --- 
Number of features: 1031
 --- ECM --- 
Number of features: 1180
 --- Melanocyte --- 
Number of features: 393


In [60]:
model_outputs = run_estimators(X=cptac_accession_numbers, stroma_model=chosen_models['Stroma'], tumor_model=chosen_models['Tumor'], lymphocyte_model=chosen_models['Lymphocyte'])
model_outputs = model_outputs.merge(cptac_molecular_subtypes, left_index=True, right_index=True)
model_outputs

NameError: name 'cptac_accession_numbers' is not defined

In [ ]:
scaffold = pd.DataFrame(columns=model_outputs.columns).drop(columns=['Stroma', 'Tumor']).rename(columns={'Lymphocyte':'Estimator Value'})
scaffold['Estimator'] = []
for x in tls_strings:
    temp = model_outputs[[x, 'prediction', 'ID', 'ProteomicSubtype']].rename(columns={x:'Estimator Value'})
    temp['Estimator'] = x
    scaffold = pd.concat([scaffold, temp])
scaffold = scaffold.reset_index()

plt.figure(figsize=(15, 8))
hue_order = tls_strings
ax = sns.boxplot(data=scaffold, x='ProteomicSubtype', y='Estimator Value', hue='Estimator', boxprops={'alpha':0.6}, hue_order=hue_order, palette=tls_colors)
sns.swarmplot(data=scaffold, x='ProteomicSubtype', y='Estimator Value', hue='Estimator', dodge=True, hue_order=hue_order, palette=tls_colors, linewidth=0.1, alpha=0.8)
ax.set_title(f'{model_type}: {validation_name} Validation')
ax.spines[['right', 'top']].set_visible(False)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=[(handles[0], handles[3]), (handles[1], handles[4]), (handles[2], handles[5])],
          labels=hue_order,
          loc='upper right', handlelength=4,
          handler_map={tuple: HandlerTuple(ndivide=None)})
          
pairs = [
    (('Mesenchymal', 'Stroma'), ('Mesenchymal', 'Tumor')),
    (('Mesenchymal', 'Stroma'), ('Mesenchymal', 'Lymphocyte')),
    (('Immunoreactive', 'Lymphocyte'), ('Immunoreactive', 'Tumor')),
    (('Immunoreactive', 'Lymphocyte'), ('Immunoreactive', 'Stroma')),
    (('Differentiated', 'Tumor'), ('Differentiated', 'Lymphocyte')),
    (('Differentiated', 'Tumor'), ('Differentiated', 'Stroma')),
    (('Proliferative', 'Tumor'), ('Proliferative', 'Lymphocyte')),
    (('Proliferative', 'Tumor'), ('Proliferative', 'Stroma')),
    (('Stromal', 'Stroma'), ('Stromal', 'Lymphocyte')),
    (('Stromal', 'Stroma'), ('Stromal', 'Tumor'))
]
annot = Annotator(ax, pairs, data=scaffold, x='ProteomicSubtype', y='Estimator Value', hue='Estimator', hue_order=hue_order)
annot.configure(test='Mann-Whitney', verbose=2, text_format='star')
annot.apply_test()
annot.annotate()

plt.savefig(f'{figure_path}/{validation_name}_validation_{model_type.replace(" ", "-").replace(",", "")}_tissue_boxplots.{file_format}', format=file_format)

plt.show()

In [ ]:
scaffold = pd.DataFrame(columns=tls_strings).T

for subtype in model_outputs.ProteomicSubtype.unique():
    temp = model_outputs[model_outputs.ProteomicSubtype == subtype]
    v_counts = temp.prediction.value_counts()
    temp_scores = {}
    for score in tls_strings:
        try:
            temp_scores[score] = v_counts[score]/v_counts.sum()
        except:
            print(f'{score} - not in {subtype} v_counts... Putting count of 0')
            temp_scores[score] = 0
    temp_scores = pd.Series(temp_scores)
    scaffold[subtype] = temp_scores
scaffold = scaffold.T
scaffold.reset_index().rename(columns={'index':'ProteomicSubtype'})
ax = scaffold.plot(kind='bar', stacked=True, figsize=(15, 8), color=tls_colors.values())
ax.set_title(f'{model_type}: CPTAC Validation')
ax.set_xlabel('ProteomicSubtype')
ax.set_ylabel('% Classified')
ax.spines[['right', 'top']].set_visible(False)

plt.xticks(rotation = 0)

plt.savefig(f'{figure_path}/{validation_name}_validation_{model_type.replace(" ", "-").replace(",", "")}_tissue_stacked_bar.{file_format}', format=file_format)

plt.show()

### ssGSEA

In [ ]:
file_names = {}
for tissue_type in tls_strings:
    model_location = f'{data_folder}/{tissue_type}/{tissue_type}_svr_sklearn_at_least_{min_num_features}_model_cptac_candidate_HGSOC.pkl'
    model = pickle.load(open(model_location, 'rb'))
    print(tissue_type, 'Model N features (genes):', len(model.feature_names_in_))

    file_name = f'{data_folder}/{tissue_type}/model_features_{tissue_type}.csv'
    pd.Series(model.feature_names_in_).to_csv(file_name)

    file_names[tissue_type] = file_name

In [ ]:
# expression file (Admixture)
expression_file = f'{data_folder}/CPTAC/020323_CPTAC_Ovarian_n174_Imputed8242Proteins_ACCESSION_NUMBERS.csv'
output_file_name = 'ssGSEA_cptac'

output_directory = f'{data_folder}/ssGSEA'
try:
    os.mkdir(output_directory)
except FileExistsError:
    print(output_directory, 'already exists...')


# run GSEA R script
process = subprocess.run(['Rscript', 'GSEA.r', expression_file, output_directory, output_file_name, file_names['Lymphocyte'], file_names['Stroma'], file_names['Tumor']])
process.check_returncode() # errors if there is a non-zero exit status

In [ ]:
subtype = pd.read_csv(f'{data_folder}/CPTAC/020323_CPTAC_Ovarian_n169_MolecularSubtypes.txt', sep='\t', index_col=1).drop(columns=['ID'])
#subtype.index = subtype.index.str.replace('-', '.')

# get colors for types
subtype_colors_dict = dict(zip(subtype.ProteomicSubtype.unique(), sns.color_palette('hls', len(subtype.ProteomicSubtype.unique()))))
subtype_colors = subtype.ProteomicSubtype.map(subtype_colors_dict)

In [ ]:
ssGSEA = pd.read_csv(f'{data_folder}/ssGSEA/{output_file_name}.csv', index_col=0)

ssGSEAt = ssGSEA.T
ssGSEAt['prediction'] = ssGSEAt.idxmax(axis=1)
ssGSEAt['ProteomicSubtype'] = subtype.ProteomicSubtype

absolute_max_ssGSEA = max(abs(ssGSEA.min().min()), abs(ssGSEA.max().max()))

# change dpi ..?
# plt.rcParams['figure.dpi'] = 200
g = sns.clustermap(data = ssGSEA, cmap='vlag', linewidth=.5, col_colors=subtype_colors, figsize=(18, 8), vmin=-absolute_max_ssGSEA, vmax=absolute_max_ssGSEA)
# custom legend
legend_elements = [Line2D([0], [0], lw=7, color=subtype_colors_dict[x], label=x) for x in subtype_colors_dict]
plt.legend(handles=legend_elements, loc=(1.6, 0.2))
plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0)

plt.savefig(f'{figure_path}/{validation_name}_ssGSEA_{model_type.replace(" ", "-").replace(",", "")}_tissue_dendro_heatmap.{file_format}', format=file_format)

plt.show()

In [ ]:
ssGSEAt = ssGSEA.T
ssGSEAt['prediction'] = ssGSEAt.idxmax(axis=1)
ssGSEAt['ProteomicSubtype'] = cptac_molecular_subtypes['ProteomicSubtype']

scaffold = pd.DataFrame(columns=ssGSEAt.columns).drop(columns=['Stroma', 'Tumor']).rename(columns={'Lymphocyte':'ssGSEA'})
scaffold['Estimator'] = []
for x in tls_strings:
    temp = ssGSEAt[[x, 'ProteomicSubtype', 'prediction']].rename(columns={x:'ssGSEA'})
    temp['Estimator'] = x
    scaffold = pd.concat([scaffold, temp])
scaffold = scaffold.reset_index()

plt.figure(figsize=(15, 8))
hue_order = tls_strings
ax = sns.boxplot(data=scaffold, x='ProteomicSubtype', y='ssGSEA', hue='Estimator', boxprops={'alpha':0.6}, hue_order=hue_order, palette=tls_colors)
sns.swarmplot(data=scaffold, x='ProteomicSubtype', y='ssGSEA', hue='Estimator', dodge=True, hue_order=hue_order, palette=tls_colors, linewidth=0.1, alpha=0.8)
ax.set_title(f'{model_type} features: CPTAC ssGSEA')
ax.spines[['right', 'top']].set_visible(False)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=[(handles[0], handles[3]), (handles[1], handles[4]), (handles[2], handles[5])],
          labels=hue_order,
          loc='upper center', handlelength=4,
          handler_map={tuple: HandlerTuple(ndivide=None)})


pairs = [
    (('Mesenchymal', 'Stroma'), ('Mesenchymal', 'Tumor')),
    (('Mesenchymal', 'Stroma'), ('Mesenchymal', 'Lymphocyte')),
    (('Immunoreactive', 'Lymphocyte'), ('Immunoreactive', 'Tumor')),
    (('Immunoreactive', 'Lymphocyte'), ('Immunoreactive', 'Stroma')),
    (('Differentiated', 'Tumor'), ('Differentiated', 'Lymphocyte')),
    (('Differentiated', 'Tumor'), ('Differentiated', 'Stroma')),
    (('Proliferative', 'Tumor'), ('Proliferative', 'Lymphocyte')),
    (('Proliferative', 'Tumor'), ('Proliferative', 'Stroma')),
    (('Stromal', 'Stroma'), ('Stromal', 'Lymphocyte')),
    (('Stromal', 'Stroma'), ('Stromal', 'Tumor'))
]
annot = Annotator(ax, pairs, data=scaffold, x='ProteomicSubtype', y='ssGSEA', hue='Estimator', hue_order=hue_order)
annot.configure(test='Mann-Whitney', verbose=2, text_format='star')
annot.apply_test()
annot.annotate()

plt.savefig(f'{figure_path}/{validation_name}_ssGSEA_{model_type.replace(" ", "-").replace(",", "")}_tissue_boxplots.{file_format}', format=file_format)

plt.show()

In [ ]:
scaffold = pd.DataFrame(columns=tls_strings).T

for subtype in ssGSEAt.ProteomicSubtype.unique():
    if str(subtype) == 'nan':
        temp = ssGSEAt[ssGSEAt.ProteomicSubtype.isna()]
    else:
        temp = ssGSEAt[ssGSEAt.ProteomicSubtype == subtype]
    v_counts = temp.prediction.value_counts()
    temp_scores = {}
    for score in tls_strings:
        try:
            temp_scores[score] = v_counts[score]/v_counts.sum()
        except:
            print(f'{score} - not in {subtype} v_counts... Putting count of 0')
            temp_scores[score] = 0
    temp_scores = pd.Series(temp_scores)
    scaffold[subtype] = temp_scores
scaffold = scaffold.T
scaffold.reset_index().rename(columns={'index':'ProteomicSubtype'})
ax = scaffold.plot(kind='bar', stacked=True, figsize=(15, 8), color=tls_colors.values())
ax.set_title(f'{model_type} features: CPTAC ssGSEA')
ax.set_xlabel('ProteomicSubtype')
ax.set_ylabel('% Classified')

plt.xticks(rotation = 0)

plt.savefig(f'{figure_path}/{validation_name}_ssGSEA_{model_type.replace(" ", "-").replace(",", "")}_tissue_stacked_bar.{file_format}', format=file_format)

plt.show()

## N9 HGSOC<a id='hgsoc'></a>

Here, I apply both the SVR models created, and ssGSEA methods to an [HGSOC dataset](https://www.sciencedirect.com/science/article/pii/S2589004221007252), containing proteomic data from ovarian cancer samples. This serves as an internal dataset for validation, with enriched tissue type being a surrogate for tissue type. 

In [ ]:
validation_name = 'HGSOC'

### Model Outputs

In [ ]:
for tissue_type in chosen_models:
    print(f' --- {tissue_type} --- ')
    print(f'Number of features: {len(chosen_models[tissue_type].feature_names_in_)}')

In [ ]:
model_outputs = run_estimators(X=hgsoc_abundance, stroma_model=chosen_models['Stroma'], tumor_model=chosen_models['Tumor'], lymphocyte_model=chosen_models['Lymphocyte'])
model_outputs = model_outputs.merge(HGSOC_enriched_type, left_index=True, right_index=True)
model_outputs

In [ ]:
# create dataframe scaffold for figure
scaffold = pd.DataFrame(columns=model_outputs.columns).drop(columns=['Stroma', 'Tumor']).rename(columns={'Lymphocyte':'Estimator Value'})
scaffold['Estimator'] = []
for x in tls_strings:
    temp = model_outputs[[x, 'prediction', 'TissueType']].rename(columns={x:'Estimator Value'})
    temp['Estimator'] = x
    scaffold = pd.concat([scaffold, temp])
scaffold = scaffold.reset_index()

# plot the figure
plt.figure(figsize=(15, 8))
hue_order = tls_strings
ax = sns.boxplot(data=scaffold, x='TissueType', y='Estimator Value', hue='Estimator', boxprops={'alpha':0.6}, hue_order=hue_order, palette=tls_colors)
sns.swarmplot(data=scaffold, x='TissueType', y='Estimator Value', hue='Estimator', dodge=True, hue_order=hue_order, palette=tls_colors, linewidth=0.1, alpha=0.8)
ax.set_title(f'{model_type}: HGSOC Validation')
ax.spines[['right', 'top']].set_visible(False)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=[(handles[0], handles[3]), (handles[1], handles[4]), (handles[2], handles[5])],
          labels=hue_order,
          loc='lower left', handlelength=4,
          handler_map={tuple: HandlerTuple(ndivide=None)})

pairs = [
    (('Enriched_Tumor', 'Tumor'), ('Enriched_Tumor', 'Stroma')),
    (('Enriched_Tumor', 'Tumor'), ('Enriched_Tumor', 'Lymphocyte')),
    (('Enriched_Stroma', 'Stroma'), ('Enriched_Stroma', 'Tumor')),
    (('Enriched_Stroma', 'Stroma'), ('Enriched_Stroma', 'Lymphocyte')),
    (('Whole_Tumor', 'Tumor'), ('Whole_Tumor', 'Stroma')),
    (('Whole_Tumor', 'Tumor'), ('Whole_Tumor', 'Lymphocyte')),
    (('Whole_Tumor', 'Stroma'), ('Whole_Tumor', 'Lymphocyte'))
]
annot = Annotator(ax, pairs, data=scaffold, x='TissueType', y='Estimator Value', hue='Estimator', hue_order=hue_order)
annot.configure(test='Mann-Whitney', verbose=2, text_format='star')
annot.apply_test()
annot.annotate()

plt.savefig(f'{figure_path}/{validation_name}_validation_{model_type.replace(" ", "-").replace(",", "")}_tissue_boxplots.{file_format}', format=file_format)

plt.show()

In [ ]:
scaffold = pd.DataFrame(columns=tls_strings).T

for subtype in model_outputs.TissueType.unique():
    temp = model_outputs[model_outputs.TissueType == subtype]
    v_counts = temp.prediction.value_counts()
    temp_scores = {}
    for score in tls_strings:
        try:
            temp_scores[score] = v_counts[score]/v_counts.sum()
        except:
            print(f'{score} - not in {subtype} v_counts... Putting count of 0')
            temp_scores[score] = 0
    temp_scores = pd.Series(temp_scores)
    scaffold[subtype] = temp_scores
scaffold = scaffold.T
scaffold.reset_index().rename(columns={'index':'TissueType'})
ax = scaffold.plot(kind='bar', stacked=True, figsize=(15, 8), color=tls_colors.values())
ax.set_title(f'{model_type}: HGSOC Validation')
ax.set_xlabel('TissueType')
ax.set_ylabel('% Classified')

plt.xticks(rotation = 0)

plt.savefig(f'{figure_path}/{validation_name}_validation_{model_type.replace(" ", "-").replace(",", "")}_tissue_stacked_bar.{file_format}', format=file_format)

plt.show()

### ssGSEA

In [ ]:
file_names = {}
for tissue_type in tls_strings:
    model_location = f'{data_folder}/{tissue_type}/{tissue_type}_svr_sklearn_at_least_{min_num_features}_model_cptac_candidate_HGSOC.pkl'
    model = pickle.load(open(model_location, 'rb'))
    print(tissue_type, 'Model N features (genes):', len(model.feature_names_in_))

    file_name = f'{data_folder}/{tissue_type}/model_features_{tissue_type}.csv'
    pd.Series(model.feature_names_in_).to_csv(file_name)

    file_names[tissue_type] = file_name

In [ ]:
# expression file (Admixture)
expression_file = f'{data_folder}/n9_HGSOC/N9_HGSOC_abundances.csv'
output_file_name = 'ssGSEA_hgsoc'

output_directory = f'{data_folder}/ssGSEA'
try:
    os.mkdir(output_directory)
except FileExistsError:
    print(output_directory, 'already exists...')


# run GSEA R script
process = subprocess.run(['Rscript', 'GSEA.r', expression_file, output_directory, output_file_name, file_names['Lymphocyte'], file_names['Stroma'], file_names['Tumor']])
process.check_returncode() # errors if there is a non-zero exit status

In [ ]:
# get colors for types
subtype_colors_dict = dict(zip(HGSOC_enriched_type.TissueType.unique(), sns.color_palette('hls', len(HGSOC_enriched_type.TissueType.unique()))))
subtype_colors = HGSOC_enriched_type.TissueType.map(subtype_colors_dict)

In [ ]:
ssGSEA = pd.read_csv(f'{data_folder}/ssGSEA/{output_file_name}.csv', index_col=0)

absolute_max_ssGSEA = max(abs(ssGSEA.min().min()), abs(ssGSEA.max().max()))

# change dpi ..?
# plt.rcParams['figure.dpi'] = 200
g = sns.clustermap(data = ssGSEA, cmap='vlag', linewidth=.5, col_colors=subtype_colors, figsize=(18, 8), vmin=-absolute_max_ssGSEA, vmax=absolute_max_ssGSEA)
# custom legend
legend_elements = [Line2D([0], [0], lw=7, color=subtype_colors_dict[x], label=x) for x in subtype_colors_dict]
plt.legend(handles=legend_elements, loc=(1.6, 0.2))
plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0)

plt.savefig(f'{figure_path}/{validation_name}_ssGSEA_{model_type.replace(" ", "-").replace(",", "")}_tissue_dendro_heatmap.{file_format}', format=file_format)

plt.show()

In [ ]:
ssGSEAt = ssGSEA.T
ssGSEAt['prediction'] = ssGSEAt.idxmax(axis=1)
ssGSEAt['EnrichedType'] = HGSOC_enriched_type['TissueType']

scaffold = pd.DataFrame(columns=ssGSEAt.columns).drop(columns=['Stroma', 'Tumor']).rename(columns={'Lymphocyte':'ssGSEA'})
scaffold['Estimator'] = []
for x in tls_strings:
    temp = ssGSEAt[[x, 'EnrichedType', 'prediction']].rename(columns={x:'ssGSEA'})
    temp['Estimator'] = x
    scaffold = pd.concat([scaffold, temp])
scaffold = scaffold.reset_index()

plt.figure(figsize=(15, 8))
hue_order = tls_strings
ax = sns.boxplot(data=scaffold, x='EnrichedType', y='ssGSEA', hue='Estimator', boxprops={'alpha':0.6}, hue_order=hue_order, palette=tls_colors)
sns.swarmplot(data=scaffold, x='EnrichedType', y='ssGSEA', hue='Estimator', dodge=True, hue_order=hue_order, palette=tls_colors, linewidth=0.1, alpha=0.8)
ax.set_title(f'{model_type} features: N9 HGSOC ssGSEA')
ax.spines[['right', 'top']].set_visible(False)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=[(handles[0], handles[3]), (handles[1], handles[4]), (handles[2], handles[5])],
          labels=hue_order,
          loc='lower left', handlelength=4,
          handler_map={tuple: HandlerTuple(ndivide=None)})

# remove legend, overlapping boxplot
legend = ax.legend()
legend.remove()

pairs = [
    (('Enriched_Tumor', 'Tumor'), ('Enriched_Tumor', 'Stroma')),
    (('Enriched_Tumor', 'Tumor'), ('Enriched_Tumor', 'Lymphocyte')),
    (('Enriched_Stroma', 'Stroma'), ('Enriched_Stroma', 'Tumor')),
    (('Enriched_Stroma', 'Stroma'), ('Enriched_Stroma', 'Lymphocyte')),
    (('Whole_Tumor', 'Tumor'), ('Whole_Tumor', 'Stroma')),
    (('Whole_Tumor', 'Tumor'), ('Whole_Tumor', 'Lymphocyte')),
    (('Whole_Tumor', 'Stroma'), ('Whole_Tumor', 'Lymphocyte'))
]
annot = Annotator(ax, pairs, data=scaffold, x='EnrichedType', y='ssGSEA', hue='Estimator', hue_order=hue_order)
annot.configure(test='Mann-Whitney', verbose=2, text_format='star')
annot.apply_test()
annot.annotate()

plt.savefig(f'{figure_path}/{validation_name}_ssGSEA_{model_type.replace(" ", "-").replace(",", "")}_tissue_boxplots.{file_format}', format=file_format)

plt.show()

In [ ]:
scaffold = pd.DataFrame(columns=tls_strings).T

for subtype in ssGSEAt.EnrichedType.unique():
    temp = ssGSEAt[ssGSEAt.EnrichedType == subtype]
    v_counts = temp.prediction.value_counts()
    temp_scores = {}
    for score in tls_strings:
        try:
            temp_scores[score] = v_counts[score]/v_counts.sum()
        except:
            print(f'{score} - not in {subtype} v_counts... Putting count of 0')
            temp_scores[score] = 0
    temp_scores = pd.Series(temp_scores)
    scaffold[subtype] = temp_scores
scaffold = scaffold.T
scaffold.reset_index().rename(columns={'index':'EnrichedType'})
ax = scaffold.plot(kind='bar', stacked=True, figsize=(15, 8), color=tls_colors.values())
ax.set_title(f'{model_type} features: N9 HGSOC ssGSEA')
ax.set_xlabel('EnrichedType')
ax.set_ylabel('% Classified')

plt.xticks(rotation = 0)

plt.savefig(f'{figure_path}/{validation_name}_ssGSEA_{model_type.replace(" ", "-").replace(",", "")}_tissue_stacked_bar.{file_format}', format=file_format)

plt.show()

# Get Gene Names
Currently, model_features_{model}.csv files have accession numbers, but make files that have their gene names.

In [ ]:
for model in tls_strings:
    in_file_name = f'data/{model}/model_features_{model}.csv'
    in_data = pd.read_csv(in_file_name)

    out_file_name = f'{in_file_name.split(".")[0]}_gene_names.csv'
    # use admixture_tissue_annotation to get gene names from accession numbers
    genes = admixture_tissue_annotation.merge(in_data, how='right', left_index=True, right_on='0')['Gene']
    assert len(genes) == in_data.shape[0] # make sure the number of genes matches the number of accession numbers
    assert len(set(genes)) == len(genes) # see if there are any repeats (there shouldn't be)
    genes.to_csv(out_file_name) # save to csv

## Convert to HTML

In [ ]:
!python -m nbconvert SVR_RFE.ipynb --to html